# **Import Library**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# **Memuat Dataset dari Hasil Clustering**

In [2]:
transaksi_clustering= pd.read_csv("https://raw.githubusercontent.com/yovelakalista23/BMLP/refs/heads/main/dataset_clustering.csv")
transaksi_clustering.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,Cluster,PCA1,PCA2
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70.0,Doctor,81,1.0,5112.21,2024-11-04 08:08:08,0,1.766409,0.247033
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68.0,Doctor,141,1.0,13758.91,2024-11-04 08:09:35,0,0.670791,1.183649
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19.0,Student,56,1.0,1122.35,2024-11-04 08:07:04,2,-0.575673,-1.465429
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26.0,Student,25,1.0,8569.06,2024-11-04 08:09:06,2,-0.460745,-1.023851
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26.0,Student,198,1.0,7429.40,2024-11-04 08:06:39,2,0.019185,-1.503782


In [3]:
transaksi_clustering.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionID            2512 non-null   object 
 1   AccountID                2512 non-null   object 
 2   TransactionAmount        2512 non-null   float64
 3   TransactionDate          2512 non-null   object 
 4   TransactionType          2512 non-null   object 
 5   Location                 2512 non-null   object 
 6   DeviceID                 2512 non-null   object 
 7   IP Address               2512 non-null   object 
 8   MerchantID               2512 non-null   object 
 9   Channel                  2512 non-null   object 
 10  CustomerAge              2512 non-null   float64
 11  CustomerOccupation       2512 non-null   object 
 12  TransactionDuration      2512 non-null   int64  
 13  LoginAttempts            2512 non-null   float64
 14  AccountBalance          

In [6]:
# Membuang kolom yang tidak digunakan
df = transaksi_clustering.drop(columns=['TransactionID', 'AccountID', 'TransactionDate', 'Location', 'DeviceID','IP Address',
                      'MerchantID', 'LoginAttempts', 'PreviousTransactionDate'], axis=1)
df.head()

,TransactionAmount,TransactionType,Channel,CustomerAge,CustomerOccupation,TransactionDuration,AccountBalance,Cluster,PCA1,PCA2
0,14.09,Debit,ATM,70.0,Doctor,81,5112.21,0,1.766409,0.247033
1,376.24,Debit,ATM,68.0,Doctor,141,13758.91,0,0.670791,1.183649
2,126.29,Debit,Online,19.0,Student,56,1122.35,2,-0.575673,-1.465429
3,184.50,Debit,Online,26.0,Student,25,8569.06,2,-0.460745,-1.023851
4,13.45,Credit,Online,26.0,Student,198,7429.40,2,0.019185,-1.503782


### **Encoding**

In [8]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
for col in ['TransactionType', 'Channel', 'CustomerOccupation']:
    df[col] = label_enc.fit_transform(df[col])


In [9]:
df.head()

,TransactionAmount,TransactionType,Channel,CustomerAge,CustomerOccupation,TransactionDuration,AccountBalance,Cluster,PCA1,PCA2
0,14.09,1,0,70.0,0,81,5112.21,0,1.766409,0.247033
1,376.24,1,0,68.0,0,141,13758.91,0,0.670791,1.183649
2,126.29,1,2,19.0,3,56,1122.35,2,-0.575673,-1.465429
3,184.50,1,2,26.0,3,25,8569.06,2,-0.460745,-1.023851
4,13.45,0,2,26.0,3,198,7429.40,2,0.019185,-1.503782


# **Data Splitting**

In [10]:
from sklearn.model_selection import train_test_split

# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['Cluster', 'PCA1', 'PCA2'])
y = df['Cluster']  # Target: 'Cluster'

# Split data menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cek bentuk data setelah split
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (2009, 7)
X_test shape: (503, 7)
y_train shape: (2009,)
y_test shape: (503,)


Memisahkan fitur X dan Y kemudian membagi dataset menjadi 2, yaitu training dataset dan testing dataset

# **Membangun model Klasifikasi**

## **a. Membangun Model Klasifikasi**

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Inisialisasi model
rf_model = RandomForestClassifier(random_state=42)
logreg_model = LogisticRegression(random_state=42, max_iter=1000)

Menggunakan algoritma Random Forest dan Logistic Regression

Kemudian Training Model

In [12]:
# Training model Random Forest
rf_model.fit(X_train, y_train)

# Training model Logistic Regression
logreg_model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

In [13]:
# Prediksi menggunakan Random Forest
y_pred_rf = rf_model.predict(X_test)

# Prediksi menggunakan Logistic Regression
y_pred_logreg = logreg_model.predict(X_test)

## **b. Evaluasi Model**

In [24]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report

# --- EVALUASI METRIK ---
# Random Forest
print("Evaluasi Random Forest:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_rf, average='weighted'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf, average='weighted'):.4f}")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_rf))

# Logistic Regression
print("Evaluasi Logistic Regression:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_logreg):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_logreg, average='weighted'):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_logreg, average='weighted'):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_logreg, average='weighted'):.4f}")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_logreg))

Evaluasi Random Forest:
Accuracy: 0.9901
F1 Score: 0.9901
Precision: 0.9901
Recall: 0.9901

Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       208
           1       0.97      0.98      0.98       104
           2       0.99      0.99      0.99       191

    accuracy                           0.99       503
   macro avg       0.99      0.99      0.99       503
weighted avg       0.99      0.99      0.99       503

Evaluasi Logistic Regression:
Accuracy: 0.9423
F1 Score: 0.9421
Precision: 0.9426
Recall: 0.9423

Classification Report:

              precision    recall  f1-score   support

           0       0.94      0.97      0.96       208
           1       0.93      0.95      0.94       104
           2       0.95      0.91      0.93       191

    accuracy                           0.94       503
   macro avg       0.94      0.94      0.94       503
weighted avg       0.94      0.94      0.94       503



Evaluasi Random Forest:
- Accuracy: 0.9901
- F1 Score: 0.9901
- Precision: 0.9901
- Recall: 0.9901

Evaluasi Logistic Regression:
- Accuracy: 0.9423
- F1 Score: 0.9421
- Precision: 0.9426
- Recall: 0.9423

## **c. Tuning Model Klasifikasi**

Menggunakan GridSearchCV karena dataset yang digunakan tidak terlalu besar, sehingga prosesnya bisa optimal dan terkontrol

In [25]:
from sklearn.model_selection import GridSearchCV

In [26]:
# Parameter grid buat Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],  # jumlah pohon
    'max_depth': [None, 10, 20, 30],  # kedalaman pohon
    'min_samples_split': [2, 5, 10],  # minimum data untuk split
    'min_samples_leaf': [1, 2, 4]     # minimum data di tiap daun
}

# Model Random Forest
rf = RandomForestClassifier(random_state=42)

# Grid Search
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf,
                              cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit model
grid_search_rf.fit(X_train, y_train)

# Hasil terbaik
print("Best Parameters for Random Forest:", grid_search_rf.best_params_)
print("Best Score for Random Forest:", grid_search_rf.best_score_)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters for Random Forest: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Score for Random Forest: 0.9850709048274835


Keterangan:

estimator=rf → model yang mau dituning.

param_grid=param_grid_rf → grid kombinasi hyperparameter.

cv=5 → pakai 5-fold cross-validation.

n_jobs=-1 → supaya pakai semua CPU core, biar cepet.

verbose=2 → biar bisa liat progress di terminal.

scoring='accuracy' → kita optimalkan accuracy.

In [27]:
# Parameter grid buat Logistic Regression
param_grid_logreg = {
    'C': [0.01, 0.1, 1, 10, 100],        # regularization strength
    'penalty': ['l1', 'l2'],              # jenis regularization
    'solver': ['liblinear']               # solver yang support l1 dan l2
}

# Model Logistic Regression
logreg = LogisticRegression(random_state=42, max_iter=1000)

# Grid Search
grid_search_logreg = GridSearchCV(estimator=logreg, param_grid=param_grid_logreg,
                                  cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit model
grid_search_logreg.fit(X_train, y_train)

# Hasil terbaik
print("Best Parameters for Logistic Regression:", grid_search_logreg.best_params_)
print("Best Score for Logistic Regression:", grid_search_logreg.best_score_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters for Logistic Regression: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score for Logistic Regression: 0.9850721455068795


Penjelasan:

param_grid=param_grid_logreg: Grid hyperparameter yang sudah disiapkan.

cv=5: 5-fold cross-validation, membagi data menjadi 5 bagian untuk validasi silang.

n_jobs=-1: Agar semua core CPU digunakan untuk mempercepat proses pencarian.

verbose=2: Agar kita bisa melihat progres grid search.

scoring='accuracy': Kita mengoptimalkan berdasarkan accuracy.

## **d. Evaluasi Model Setelah Melakukan Tuning**

In [28]:
best_model_rf = grid_search_rf.best_estimator_  # Random Forest
best_model_logreg = grid_search_logreg.best_estimator_  # Logistic Regression

In [29]:
y_pred_rf_tuned = best_model_rf.predict(X_test)  # Prediksi dengan Random Forest
y_pred_logreg_tuned = best_model_logreg.predict(X_test)  # Prediksi dengan Logistic Regression

In [30]:
accuracy_rf_tuned = accuracy_score(y_test, y_pred_rf_tuned)  # Random Forest
f1_rf_tuned = f1_score(y_test, y_pred_rf_tuned, average='weighted')  # Random Forest
precision_rf_tuned = precision_score(y_test, y_pred_rf_tuned, average='weighted')
recall_rf_tuned = recall_score(y_test, y_pred_rf_tuned, average='weighted')

f1_logreg_tuned = f1_score(y_test, y_pred_logreg_tuned, average='weighted')  # Logistic Regression
accuracy_logreg_tuned = accuracy_score(y_test, y_pred_logreg_tuned)  # Logistic Regression
precision_logreg_tuned = precision_score(y_test, y_pred_logreg_tuned, average='weighted')
recall_logreg_tuned = recall_score(y_test, y_pred_logreg_tuned, average='weighted')


In [31]:
print(f"Random Forest Accuracy (tuned): {accuracy_rf_tuned:.4f}")
print(f"Random Forest F1-Score (tuned): {f1_rf_tuned:.4f}")
print(f"Random Forest Precision (tuned): {precision_rf_tuned:.4f}")
print(f"Random Forest Recall (tuned): {recall_rf_tuned:.4f}")

Random Forest Accuracy (tuned): 0.9901
Random Forest F1-Score (tuned): 0.9901
Random Forest Precision (tuned): 0.9901
Random Forest Recall (tuned): 0.9901


In [32]:
print(f"Logistic Regression Accuracy (tuned): {accuracy_logreg_tuned:.4f}")
print(f"Logistic Regression F1-Score (tuned): {f1_logreg_tuned:.4f}")
print(f"Logistic Regression Precision (tuned): {precision_logreg_tuned:.4f}")
print(f"Logistic Regression Recall (tuned): {recall_logreg_tuned:.4f}")

Logistic Regression Accuracy (tuned): 0.9821
Logistic Regression F1-Score (tuned): 0.9820
Logistic Regression Precision (tuned): 0.9824
Logistic Regression Recall (tuned): 0.9821


## **e. Analisis Hasil Evaluasi Setelah Tuning**

Peningkatan Kinerja:

Setelah tuning, salah satu model mengalami peningkatan performa yang signifikan.

**Random Forest tidak mengalami peningkatan di semua metrik:**

- Accuracy tetap 0.9901.

- F1-Score tetap 0.9901.

- Precision tetap 0.9901.

- Recall tetap 0.9901.

**Logistic Regression menunjukkan peningkatan:**

- Accuracy meningkat dari 0.9423 menjadi 0.9821.

- F1-Score meningkat dari 0.9421 menjadi 0.9820.

- Precision meningkat dari 0.9426 menjadi 0.9824.

- Recall meningkat dari 0.9423 menjadi 0.9821.



Impak Tuning Hyperparameter:

Tuning hyperparameter jelas memberikan peningkatan yang signifikan dalam salah satu model yaitu Logistic Regression. Ini membuktikan pentingnya optimasi parameter untuk mencapai hasil terbaik. Tetapi dalam Random Forest tidak adanya peningkatan dari sebelum tuning, yaitu hasilnya sama.

Logistic Regression vs Random Forest:

Random Forest memiliki Accuracy, F1-Score, Precision, dan Recall yang sedikit lebih tinggi dibandingkan Logistic Regression setelah tuning.

Ini menunjukkan bahwa Random Forest lebih optimal untuk dataset ini setelah tuning meskipun tidak ada peningkatan yang signifikan, meskipun keduanya menunjukkan performa yang sangat baik.